## Example 4. The Perturbed Kepler's problem

This notebook solves the perturbed Kepler's problem by different RK methods and their relaxation versions:

\begin{align}
    \dot{q_{1}} & = p_{1} \\
    \dot{q_{2}} & = p_{2} \\
    \dot{p_{1}} & = -\frac{q_{1}}{\left(q_{1}^2+q_{2}^2\right)^{3/2}} -\mu \frac{q_{1}}{\left(q_{1}^2+q_{2}^2\right)^{5/2}} \\
    \dot{p_{2}} & =-\frac{q_{2}}{\left(q_{1}^2+q_{2}^2\right)^{3/2}}  -\mu\frac{q_{2}}{\left(q_{1}^2+q_{2}^2\right)^{5/2}} \;,
\end{align}
where $\mu = 0.005$ and the initial condition is $ \left(q_{1}(0),q_{2}(0),p_{1}(0),p_{2}(0)\right)^{T}=\left(1-e,0,0,\sqrt{\frac{1+e}{1-e}}\right)^{T}$, where $e=0.6$. 
Two conserved quantities are:
\begin{align}
    H(q,p) & = \frac{1}{2}\left(p_{1}^2+p_{2}^2\right)-\frac{1}{\sqrt{q_{1}^2+q_{2}^2}}-\frac{\mu}{2\sqrt{\left(q_{1}^2+q_{2}^2\right)^3}}\ (\text{Hamiltonian}) \\
    L(q,p) & = q_{1}p_{2}-q_{2}p_{1}\ (\text{angular momentum}) \;.
\end{align}

In [ ]:
#Required libraries 
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from nodepy import rk
from IPython.display import clear_output
from RKSchemes import ssp22, heun33, ssp33, rk44, fehlberg45, dp75

In [ ]:
# Parameter 
mu = 0.005

# Required functions for Kepler's problem
def kepler_f(u):
    q1 = u[0]
    q2 = u[1]
    p1 = u[2]
    p2 = u[3]
    abs_q = np.sqrt(q1*q1 + q2*q2)   
    dq1 = p1
    dq2 = p2
    dp1 = -q1 / (abs_q*abs_q*abs_q) -(3*mu/2)*(q1 / (abs_q*abs_q*abs_q*abs_q*abs_q))
    dp2 = -q2 / (abs_q*abs_q*abs_q) -(3*mu/2)*(q2 / (abs_q*abs_q*abs_q*abs_q*abs_q))
    return np.array([dq1, dq2, dp1, dp2])

def kepler_energy_H(u):
    abs_q = np.sqrt(u[0]*u[0] + u[1]*u[1])
    abs2_q = u[0]*u[0] + u[1]*u[1]
    abs2_p = u[2]*u[2] + u[3]*u[3]
    return 0.5 * abs2_p - 1.0 / np.sqrt(abs2_q) - (mu/2)*(1.0/(abs_q*abs_q*abs_q))

def kepler_angular_momentum_L(u):
    q1 = u[0]
    q2 = u[1]
    p1 = u[2]
    p2 = u[3]
    return q1*p2 - q2*p1

def rgam(gammas,u,inc1,inc2,E1_old,E2_old):
    gamma1, gamma2 = gammas
    uprop = u + gamma1*inc1 + gamma2*inc2  
    E1 = kepler_energy_H(uprop)
    E2 = kepler_angular_momentum_L(uprop)
    return np.array([E1-E1_old,E2-E2_old])

### Baseline RK methods

In [ ]:
# Compute solution with baseline methods
def compute_sol_without_relaxation(Mthdname,rkm, dt, f, T, u0,t0): 
    tt = np.zeros(1) 
    t = t0; tt[0] = t
    uu = np.zeros((1,np.size(u0))) 
    uu[0,:] = u0.copy()
    
    s = len(rkm)
    y = np.zeros((s,len(u0))) 
    F = np.zeros((s,len(u0))) 
    steps = 0
    
    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        if t + dt > T:
            dt = T - t
        for i in range(s):
            y[i,:] = uu[-1].copy()
            for j in range(i):
                y[i,:] += dt*rkm.A[i,j]*F[j,:]
            F[i,:] = f(y[i,:])
        inc = dt*sum([rkm.b[i]*F[i] for i in range(s)])    
        unew = uu[-1]+inc; t+= dt
        tt = np.append(tt, t)
        steps +=1
        uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0)  
        print("Method = Baseline %s: Step number = %d (time = %1.2f)"%(Mthdname,steps,tt[-1]))
    return tt, uu

### Relaxation RK methods

In [ ]:
# Computing solution with multiple relaxation methods
def compute_sol_multi_relaxation(Mthdname, rkm, dt, f, T, u0, t0):
    tt = np.zeros(1) 
    t = 0; tt[0] = t
    uu = np.zeros((1,np.size(u0))) 
    uu[0,:] = u0.copy()
    
    s = len(rkm)
    y = np.zeros((s,len(u0))) 
    F = np.zeros((s,len(u0))) 
    
    G1 = np.array([]); G2 = np.array([]) 
    no_inv = 2; gammas0 = np.zeros(no_inv)
    
    errs = 0; steps = 0
    
    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        if t + dt > T:
            dt = T - t
        for i in range(s):
            y[i,:] = uu[-1].copy()
            for j in range(i):
                y[i,:] += dt*rkm.A[i,j]*F[j,:]
            F[i,:] = f(y[i,:])
            
        inc1 = dt*sum([rkm.b[i]*F[i] for i in range(s)])
        inc2 = dt*sum([rkm.bhat[i]*F[i] for i in range(s)])
        
        wr_unew = uu[-1] + inc1
        E1_old = kepler_energy_H(uu[-1]); E2_old = kepler_angular_momentum_L(uu[-1])
        
        gammas, info, ier, mesg = fsolve(rgam,gammas0,args=(wr_unew,inc1,inc2,E1_old,E2_old),full_output=True)
        gamma1, gamma2= gammas
        
        steps += 1
        unew =  wr_unew + gamma1*inc1 + gamma2*inc2; t+=(1+gamma1+gamma2)*dt
        tt = np.append(tt, t)
        uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0) 
        G1 = np.append(G1, gamma1); G2 = np.append(G2, gamma2)
        print("Method = Relaxation %s: At step number = %d (time = %1.2f), integer flag = %d and γ1+γ2 = %f \n"%(Mthdname,steps,tt[-1],ier,gamma1+gamma2))
    
    return tt, uu, G1, G2

### Compute solution by all methods

In [ ]:
# Computing solutions by all the methods
%time 
eqn = 'Per_Kep'
methods = [ssp33,fehlberg45,dp75]
method_labels = ["SSPRK(3,3)", "Fehlberg(6,4)", "DP5(7,5)"]
method_names = ["SSPRKs3p3", "Fehlbergs6p4", "DP5s7p5"]

# Inputs to solve the system of ODEs
DT = [ .05,.05,.1];  f = kepler_f; T = 300; 
# Initial condition
e = 0.6; t0 = 0; u0 = np.array([1.0 - e, 0.0, 0.0, np.sqrt((1+e)/(1-e))])

b_tt = []; b_uu = []; r_tt = []; r_uu = []  

for idx in range(len(methods)):
    print(idx)
    rkm = methods[idx]; dt = DT[idx]
    tt, uu, G1, G2 = compute_sol_multi_relaxation(method_labels[idx], rkm, dt, f, T, u0, t0)
    r_tt.append(tt); r_uu.append(uu)    
    
for idx in range(len(methods)):
    rkm = methods[idx]; dt = DT[idx]
    tt, uu = compute_sol_without_relaxation(method_labels[idx], rkm, dt, f, T, u0, t0)
    b_tt.append(tt); b_uu.append(uu)


In [ ]:
import os
path = '%s'%('Figures')

import os
if not os.path.exists(path):
   os.makedirs(path)

### Compute and plot change in invariants by different methods

In [ ]:
b_INV_H = []; b_INV_L = []
r_INV_H = []; r_INV_L = []
for i in range(len(methods)):
    b_inv_H = [kepler_energy_H(u) for u in b_uu[i]] - kepler_energy_H(b_uu[i][0])
    b_inv_L = [kepler_angular_momentum_L(u) for u in b_uu[i]] - kepler_angular_momentum_L(b_uu[i][0])
    r_inv_H = [kepler_energy_H(u) for u in r_uu[i]] - kepler_energy_H(r_uu[i][0])
    r_inv_L = [kepler_angular_momentum_L(u) for u in r_uu[i]] - kepler_angular_momentum_L(r_uu[i][0])
    b_INV_H.append(b_inv_H); b_INV_L.append(b_inv_L)
    r_INV_H.append(r_inv_H); r_INV_L.append(r_inv_L)

In [ ]:
font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font) 
plt.figure(figsize=(15, 4))
for i in range(len(methods)):
    plt.subplot(1,3,i+1)
    plt.plot(b_tt[i],b_INV_H[i],':r',label="Baseline: $H(p(t),q(t))-H(p(0),q(0))$")
    plt.plot(r_tt[i],r_INV_H[i],'-r',label="Relaxation: $H(p(t),q(t))-H(p(0),q(0))$")
    plt.plot(b_tt[i],b_INV_L[i],':b',label="Baseline: $L(p(t),q(t))-L(p(0),q(0))$")
    plt.plot(r_tt[i],r_INV_L[i],'-b',label="Relaxation: $L(p(t),q(t))-L(p(0),q(0))$")
    plt.title("%s with $\Delta t$ = %.2f"%(method_labels[i],DT[i]))
    plt.xlabel("$t$");
    plt.yscale("symlog", linthresh=1.e-14)
    plt.yticks([-1.e-6, -1.e-10, -1.e-14, 1.e-14, 1.e-10, 1.e-6])
    plt.tight_layout()
    
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=2, bbox_to_anchor=(0.5, 1.2))
plt.savefig("./Figures/Per_Kepler_2_Inv_Error_Time.pdf", bbox_inches="tight")

### Proxy of Exact solution by Dense Output of "dopri5" method

In [ ]:
import numpy as np
from scipy import integrate 
from scipy.integrate import ode
# true solution by Dormand–Prince method 

def Per_Kepler_f_tu(t,u):
    q1 = u[0]
    q2 = u[1]
    p1 = u[2]
    p2 = u[3]
    abs_q = np.sqrt(q1*q1 + q2*q2)   
    dq1 = p1
    dq2 = p2
    dp1 = -q1 / (abs_q*abs_q*abs_q) -(3*mu/2)*(q1 / (abs_q*abs_q*abs_q*abs_q*abs_q))
    dp2 = -q2 / (abs_q*abs_q*abs_q) -(3*mu/2)*(q2 / (abs_q*abs_q*abs_q*abs_q*abs_q))
    return np.array([dq1, dq2, dp1, dp2])

def analytical_u_Per_Kep(t,u0):
    true_u = np.zeros((len(t), len(u0)))   
    t0 = 0; true_u[0, :] = u0
    r = integrate.ode(Per_Kepler_f_tu).set_integrator("dopri5", rtol=1e-16, atol=1e-16)
    r.set_initial_value(u0, t0)
    for i in range(1, t.size):
        true_u[i, :] = r.integrate(t[i]) 
        if not r.successful():
            raise RuntimeError("Could not integrate")
    return true_u

### Compute and plot errors by different methods

In [ ]:
b_ERR = []; r_ERR = []; B_uexact = []; R_uexact = []
# Initial condition
e = 0.6; u0 = np.array([1.0 - e, 0.0, 0.0, np.sqrt((1+e)/(1-e))])
for i in range(len(methods)):
    # maximum norm
    b_t = b_tt[i]; b_uexact = analytical_u_Per_Kep(b_t,u0)
    b_err = np.max(np.abs(b_uu[i]-b_uexact),axis=1)  
    r_t = r_tt[i]; r_uexact = analytical_u_Per_Kep(r_t,u0)
    r_err = np.max(np.abs(r_uu[i]-r_uexact),axis=1)
    b_ERR.append(b_err); r_ERR.append(r_err)
    B_uexact.append(b_uexact); R_uexact.append(r_uexact)   

In [ ]:
lgd_box_pos = [[0.4,0.2,0.5, 0.5],[0.4,0.1,0.5, 0.5],[0.4,0.5,0.5, 0.5]]
sl1_cons_mult = [1e-3,6e-6,1.5e-5]; sl1_p = [1,1,1]
sl2_cons_mult = [4*1e-4,6*1e-7,1e-5]; sl2_p = [2,2,2]
y_scale_line = [1e-10,1e-8,1e-7]
font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font)
shift = 1
plt.figure(figsize=(15, 4))

for i in range(len(methods)):
    plt.subplot(1,3,i+1)
    plt.plot(b_tt[i]+shift,b_ERR[i],':',color='orangered',label="Baseline")
    plt.plot(r_tt[i]+shift,r_ERR[i],'-g',label="Relaxation")
    sl_t = np.linspace(10,T,1000)
    sl_t_1 = np.linspace(10,50,1000)

    if i !=0:
        plt.plot(sl_t,sl1_cons_mult[i]*sl_t**sl1_p[i],'--',color='0.5',label="$\mathcal{O}(t^{%d})$"%(sl1_p[i]))
        plt.plot(sl_t,sl2_cons_mult[i]*sl_t**sl2_p[i],'-',color='0.5',label="$\mathcal{O}(t^{%d})$"%(sl2_p[i]))
    else:
        plt.plot(sl_t,sl1_cons_mult[i]*sl_t**sl1_p[i],'--',color='0.5',label="$\mathcal{O}(t^{%d})$"%(sl1_p[i]))
        plt.plot(sl_t_1,sl2_cons_mult[i]*sl_t_1**sl2_p[i],'-',color='0.5',label="$\mathcal{O}(t^{%d})$"%(sl2_p[i]))

        
    plt.xscale("log"); plt.yscale("log")
    plt.xlabel('t'); plt.ylabel('Error in q')
    plt.title("%s with $\Delta t$ = %.2f"%(method_labels[i],DT[i]))
    plt.tight_layout()
    
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=6, bbox_to_anchor=(0.5, 1.1))
plt.savefig("./Figures/Per_Kepler_Sol_Err_Time.pdf", bbox_inches="tight")

### Plot numerical solutions 

In [ ]:
colors = ["red", "blue"]
method_labels = ["SSPRK(3,3)", "Fehlberg(6,4)", "DP5(7,5)"]
for i in range(len(methods)):
    fig1 = plt.figure(i,figsize=(4,4))
    plt.plot(b_uu[i][:,0],b_uu[i][:,1],'-r',label = "Baseline: %s"%(method_labels[i]))
    plt.xlabel("$q_{1}$");  plt.ylabel("$q_{2}$")
    plt.legend()
    fig1.tight_layout()
    plt.savefig('./Figures/Per_Kepler_Baseline_%s_Sol_dt%1.0e_tf%d.pdf'%(method_names[i],DT[i],T),format='pdf', bbox_inches="tight",transparent=True)
    
for j in range(len(methods)):
    fig2 = plt.figure(3+j,figsize=(4,4))
    plt.plot(r_uu[j][:,0],r_uu[j][:,1],'-b',label = "Relaxation: %s"%(method_labels[j]))
    plt.xlabel("$q_{1}$");  plt.ylabel("$q_{2}$")
    plt.legend()
    fig2.tight_layout()
    plt.savefig('./Figures/Per_Kepler_Relaxation_%s_Sol_dt%1.0e_tf%d.pdf'%(method_names[j],DT[j],T),format='pdf', bbox_inches="tight",transparent=True)


### Plotting exact solution and comparision with numerical solution at a few last time steps

In [ ]:
# Compute exact solution 
TT = np.linspace(0,T,10000)
delta_t = TT[1]-TT[0]
Exact_Sol = analytical_u_Per_Kep(TT,u0) 

In [ ]:
colors = ["red", "blue"]; idx = 50
method_labels = ["SSPRK(3,3)", "Fehlberg(6,4)", "DP(7,5)"]
method_names = ["SSPRKs3p3", "Fehlbergs6p4", "DP5s7p5"]

# Font size    
font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font) 

for i in range(len(methods)):
    fig = plt.figure(i,figsize=(4,4))
    plt.plot(b_uu[i][-idx:-1,0],b_uu[i][-idx:-1,1],'o r',label = "Baseline: %s"%(method_labels[i]))
    plt.plot(B_uexact[i][-idx:-1,0],B_uexact[i][-idx:-1,1],'-k',label = "Exact solution",linewidth=3)
    plt.plot(r_uu[i][-idx:-1,0],r_uu[i][-idx:-1,1],'o b',label = "Relaxation: %s"%(method_labels[i]))
    plt.legend()
    fig.tight_layout()
    plt.savefig('./Figures/Per_Kepler_%s_LastFewStepsSolComp_tf%d.pdf'%(method_names[i],T),format='pdf', bbox_inches="tight",transparent=True)
    
# plot exact solution
plt.figure(figsize=(4,4))
plt.plot(Exact_Sol [:,0],Exact_Sol[:,1],'-k',label = "Exact Solution")
plt.legend()
fig.tight_layout()
plt.savefig('./Figures/Per_Kepler_Exact_Orbit_dt%1.0e_tf%d.pdf'%(delta_t,T),format='pdf', bbox_inches="tight",transparent=True)